In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [54]:
out='../output/'
inp='../input/monthly/'
pref='fpw'

In [55]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
144,(2022/02/06) 23-24,29.94,22
145,Min,29.94,0
146,Max,170,22
147,Average,76.684861111111111111111111111,11.152777777777777777777777778


In [56]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
140,(2022/02/06) 19-20,29.94,22.0
141,(2022/02/06) 20-21,29.94,22.0
142,(2022/02/06) 21-22,29.94,22.0
143,(2022/02/06) 22-23,29.94,22.0


In [57]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
140,(2022/02/06) 19-20,29.94,22.0,2022-02-06,19
141,(2022/02/06) 20-21,29.94,22.0,2022-02-06,20
142,(2022/02/06) 21-22,29.94,22.0,2022-02-06,21
143,(2022/02/06) 22-23,29.94,22.0,2022-02-06,22


In [58]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
140,29.94,22.0,2022-02-06,19
141,29.94,22.0,2022-02-06,20
142,29.94,22.0,2022-02-06,21
143,29.94,22.0,2022-02-06,22


In [59]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2022-02-06,29.94,22.0,19,2022,2,6,6
2022-02-06,29.94,22.0,20,2022,2,6,6
2022-02-06,29.94,22.0,21,2022,2,6,6


In [60]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()
daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 3     6     200.0   5800.00            29.00      97.21      29.00   
           7     400.0  11600.00            29.00      97.21      29.00   
           8     400.0  11600.00            29.00      97.21      29.00   
           9     400.0  11600.00            29.00      97.21      29.00   
           10    200.0   5800.00            29.00      97.21      29.00   
...                ...       ...              ...        ...        ...   
2022 2     2     176.0   5269.44            29.94     170.00      29.94   
           3     176.0   5269.44            29.94     170.00      29.94   
           4     176.0   5269.44            29.94     170.00      29.94   
           5     286.0   8562.84            29.94     107.00      29.94   
           6     528.0  15808.32            29.94      29.94      29.94   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 3     6     65.852500       50.0        0.0   10.000000   
           7     59.710417       50.0        0.0   16.666667   
           8     59.710417       50.0        0.0   16.666667   
           9     59.710417       50.0        0.0   16.666667   
           10    65.852500       50.0        0.0   10.000000   
...                    ...        ...        ...         ...   
2022 2     2     94.438333       22.0        0.0    7.333333   
           3     94.438333       22.0        0.0    7.333333   
           4     94.438333       22.0        0.0    7.333333   
           5     56.911000       22.0        0.0   14.300000   
           6     29.940000       22.0       22.0   22.000000   

                price_morning_max  price_evening_max  price_standard  \
year month day                                                         
2017 3     6                97.21              97.21           65.00   
           7                97.21              97.21           65.00   
           8                97.21              97.21           65.00   
           9                97.21              97.21           65.00   
           10               97.21              97.21           65.00   
...                           ...                ...             ...   
2022 2     2               170.00             170.00          107.00   
           3               170.00             170.00          107.00   
           4               170.00             170.00          107.00   
           5               107.00             107.00           29.94   
           6                29.94              29.94           29.94   

                     date  
year month day             
2017 3     6   2017-03-06  
           7   2017-03-07  
           8   2017-03-08  
           9   2017-03-09  
           10  2017-03-10  
...                   ...  
2022 2     2   2022-02-02  
           3   2022-02-03  
           4   2022-02-04  
           5   2022-02-05  
           6   2022-02-06  

[1745 rows x 13 columns]

In [61]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.00,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.00,standard
2017-03-06,65.00,0.0,6,2017,3,6,0,0.00,morning
2017-03-06,97.21,0.0,7,2017,3,6,0,0.00,morning
2017-03-06,97.21,0.0,8,2017,3,6,0,0.00,morning
...,...,...,...,...,...,...,...,...,...
2022-02-06,29.94,22.0,19,2022,2,6,6,658.68,off
2022-02-06,29.94,22.0,20,2022,2,6,6,658.68,off
2022-02-06,29.94,22.0,21,2022,2,6,6,658.68,off


In [62]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 3     6     200.0   5800.00            29.00      97.21      29.00   
           7     400.0  11600.00            29.00      97.21      29.00   
           8     400.0  11600.00            29.00      97.21      29.00   
           9     400.0  11600.00            29.00      97.21      29.00   
           10    200.0   5800.00            29.00      97.21      29.00   
...                ...       ...              ...        ...        ...   
2022 2     2     176.0   5269.44            29.94     170.00      29.94   
           3     176.0   5269.44            29.94     170.00      29.94   
           4     176.0   5269.44            29.94     170.00      29.94   
           5     286.0   8562.84            29.94     107.00      29.94   
           6     528.0  15808.32            29.94      29.94      29.94   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 3     6     65.852500       50.0        0.0   10.000000   
           7     59.710417       50.0        0.0   16.666667   
           8     59.710417       50.0        0.0   16.666667   
           9     59.710417       50.0        0.0   16.666667   
           10    65.852500       50.0        0.0   10.000000   
...                    ...        ...        ...         ...   
2022 2     2     94.438333       22.0        0.0    7.333333   
           3     94.438333       22.0        0.0    7.333333   
           4     94.438333       22.0        0.0    7.333333   
           5     56.911000       22.0        0.0   14.300000   
           6     29.940000       22.0       22.0   22.000000   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2017 3     6                97.21  ...                 3.0   
           7                97.21  ...                 3.0   
           8                97.21  ...                 3.0   
           9                97.21  ...                 3.0   
           10               97.21  ...                 3.0   
...                           ...  ...                 ...   
2022 2     2               170.00  ...                 3.0   
           3               170.00  ...                 3.0   
           4               170.00  ...                 3.0   
           5               107.00  ...                 NaN   
           6                29.94  ...                 NaN   

                evening_hour_count off_volume  standard_volume  \
year month day                                                   
2017 3     6                   2.0      150.0             50.0   
           7                   2.0      350.0             50.0   
           8                   2.0      350.0             50.0   
           9                   2.0      350.0             50.0   
           10                  2.0      150.0             50.0   
...                            ...        ...              ...   
2022 2     2                   2.0      154.0             22.0   
           3                   2.0      154.0             22.0   
           4                   2.0      154.0             22.0   
           5                   NaN      286.0              0.0   
           6                   NaN      528.0              NaN   

                morning_volume  evening_volume  off_value  standard_value  \
year month day                                                              
2017 3     6               0.0             0.0    4350.00         1450.00   
           7               0.0             0.0   10150.00         1450.00   
           8               0.0             0.0   10150.00         1450.00   
           9               0.0             0.0   10150.00         1450.00   
           10              0.0             0.0    4350.00         1450.00   
..

In [63]:
daily.to_csv(out + pref + '_daily.csv')

In [64]:
hourly.to_csv(out + pref + '_hourly.csv')

In [65]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-03-06,4,29.00,50.0,off
2017-03-06,5,29.00,50.0,standard
2017-03-06,6,65.00,0.0,morning
2017-03-06,7,97.21,0.0,morning
2017-03-06,8,97.21,0.0,morning
...,...,...,...,...
2022-02-06,19,29.94,22.0,off
2022-02-06,20,29.94,22.0,off
2022-02-06,21,29.94,22.0,off


In [66]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [67]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,587.105263,18165.002632,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.939942,27.459177,34.148967,64.340852,86.158317,41.512195,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,949.871429,42241.205667,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.470446,26.864611,47.257466,79.193597,88.723370,41.300000,50.655556,34.235088,60.633333
3,2017,6,477.653846,23265.706154,71.232500,29.95,58.039212,28.461538,10.423077,20.528205,...,40.0,48.708299,33.990683,61.625498,86.514416,115.321589,24.819788,11.700000,19.868778,7.550000
4,2017,7,460.096774,20449.032097,70.355000,25.05,52.809522,22.838710,11.580645,20.202016,...,42.0,44.445067,32.064852,59.895533,79.061769,110.973696,24.547486,10.317460,18.755102,5.476190
5,2017,8,455.000000,22275.962903,64.280500,28.61,52.141602,23.419355,14.354839,19.951882,...,46.0,48.958160,32.666806,57.921955,82.467739,99.018571,21.492537,16.405797,19.771318,14.608696
6,2017,9,434.916667,19724.003333,63.146000,26.93,50.934131,21.000000,12.583333,18.947917,...,42.0,45.351224,31.534111,57.501670,75.761157,94.614187,21.301775,12.619048,19.091837,8.928571
7,2017,10,629.367742,29708.751226,60.922500,26.92,48.947333,29.032258,22.554839,27.563871,...,44.0,47.204121,31.441835,58.422616,76.447836,88.679545,28.891429,22.969697,28.235484,20.000000
8,2017,11,657.756667,28031.525633,59.438500,26.14,48.352639,38.596667,22.306667,28.336250,...,44.0,42.616863,28.747793,55.714666,76.915099,85.168890,35.936970,22.227273,21.691532,23.340909
9,2017,12,621.906452,28655.456194,64.220000,26.61,49.656339,32.809677,20.435484,27.004651,...,42.0,46.076795,31.036576,59.088798,78.054909,89.241905,30.109497,21.920635,25.628163,20.000000


In [68]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,587.105263,18165.002632,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.939942,27.459177,34.148967,64.340852,86.158317,41.512195,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,949.871429,42241.205667,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.470446,26.864611,47.257466,79.193597,88.723370,41.300000,50.655556,34.235088,60.633333
3,2017,6,477.653846,23265.706154,71.232500,29.95,58.039212,28.461538,10.423077,20.528205,...,40.0,48.708299,33.990683,61.625498,86.514416,115.321589,24.819788,11.700000,19.868778,7.550000
4,2017,7,460.096774,20449.032097,70.355000,25.05,52.809522,22.838710,11.580645,20.202016,...,42.0,44.445067,32.064852,59.895533,79.061769,110.973696,24.547486,10.317460,18.755102,5.476190
5,2017,8,455.000000,22275.962903,64.280500,28.61,52.141602,23.419355,14.354839,19.951882,...,46.0,48.958160,32.666806,57.921955,82.467739,99.018571,21.492537,16.405797,19.771318,14.608696
6,2017,9,434.916667,19724.003333,63.146000,26.93,50.934131,21.000000,12.583333,18.947917,...,42.0,45.351224,31.534111,57.501670,75.761157,94.614187,21.301775,12.619048,19.091837,8.928571
7,2017,10,629.367742,29708.751226,60.922500,26.92,48.947333,29.032258,22.554839,27.563871,...,44.0,47.204121,31.441835,58.422616,76.447836,88.679545,28.891429,22.969697,28.235484,20.000000
8,2017,11,657.756667,28031.525633,59.438500,26.14,48.352639,38.596667,22.306667,28.336250,...,44.0,42.616863,28.747793,55.714666,76.915099,85.168890,35.936970,22.227273,21.691532,23.340909
9,2017,12,621.906452,28655.456194,64.220000,26.61,49.656339,32.809677,20.435484,27.004651,...,42.0,46.076795,31.036576,59.088798,78.054909,89.241905,30.109497,21.920635,25.628163,20.000000


In [69]:
monthly=monthly.set_index(['year','month'])

In [70]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [71]:
monthly.to_csv(out + pref + '_monthly.csv')

In [72]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [73]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,price_morning_max,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,12962.407490,47.726034,111.316903,37.184000,67.174736,19.912258,4.410968,11.864382,111.316903,...,76.40,401834.6322,344.2,252.8,66.6,44.4,16.083803,9.574360,4.314148,1.669317
1,2,10261.593875,60.350376,117.119732,39.919375,75.439489,14.533929,2.177679,7.824018,117.119732,...,0.00,229859.7028,256.8,181.8,47.4,31.6,12.552805,4.937696,1.376111,0.000000
2,3,10578.291888,43.741197,109.787832,36.123916,64.948604,20.606993,4.020979,11.188980,109.787832,...,40.40,302539.1480,318.2,234.0,61.8,41.2,20.900574,5.659563,2.424076,1.346667
3,4,14322.235726,39.787001,99.564194,31.688468,58.030657,23.585484,5.634677,14.456647,99.564194,...,217.88,355191.4460,278.0,201.8,53.4,35.6,25.188274,15.507457,14.741948,14.376494
4,5,16199.287828,37.251299,96.938414,29.579103,55.420767,29.296552,8.718621,17.349391,96.938414,...,590.96,469779.3470,325.6,235.4,61.8,41.2,23.451934,13.820844,15.504348,17.285963
5,6,19375.448548,43.268905,132.180068,34.696849,70.872123,36.750685,6.981507,20.099789,132.180068,...,63.00,565763.0976,326.8,239.4,63.0,42.0,31.348995,11.148157,4.169149,1.571905
6,7,17973.679252,46.633799,140.012387,37.616774,78.045525,30.968387,5.889032,16.488984,140.012387,...,171.40,557184.0568,344.2,252.8,66.6,44.4,27.301693,7.688665,4.572188,3.945238
7,8,19595.761845,52.794395,137.784065,37.434065,74.459988,28.167742,6.705806,16.802661,137.784065,...,333.20,607468.6172,344.2,252.8,66.6,44.4,23.561511,11.511842,8.316864,7.243478
8,9,23824.063513,48.985359,108.100267,33.143133,62.899446,39.917333,8.128000,21.818361,108.100267,...,383.00,714721.9054,339.2,242.8,63.6,42.4,32.017243,13.026594,10.048384,9.481667
9,10,16114.433348,47.814364,93.753806,33.464452,61.060316,19.440645,7.306452,14.489763,93.753806,...,214.00,499547.4338,344.2,252.8,66.6,44.4,17.824633,13.428743,7.825692,4.826087


In [74]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [75]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      37.218091  17.035455   656.716500
      1      37.218091  17.035455   656.716500
      2      37.218091  17.035455   656.716500
      3      37.218091  17.035455   656.716500
      4      37.184000  17.017419   657.171090
...                ...        ...          ...
12    19    126.787806  12.840000   725.605355
      20     71.233355  21.669032  1190.360852
      21     71.233355  21.669032  1190.360852
      22     38.137871  42.378710  1755.612090
      23     38.137871  42.378710  1755.612090

[288 rows x 3 columns]

In [76]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [77]:
from statsmodels.tsa.stattools import pacf

In [78]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.726893,0.719048,0.730482,0.825947
2,-0.036796,-0.174392,-0.159377,-0.282579
3,0.155861,0.300957,0.285549,0.112552
4,0.239361,0.203196,0.236393,0.200566
5,0.040033,0.186940,0.147484,0.021267
6,0.100729,0.109491,0.153116,0.001463
7,-0.062432,-0.172870,-0.200100,-0.117453
8,-0.139265,0.033238,0.006624,0.042629
9,-0.285533,-0.373112,-0.392633,-0.254326


In [79]:
pacfs.to_csv(out + pref + '_pacfs.csv')